In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
import bpy
from mathutils import Vector

In [2]:
mp_pose = mp.solutions.pose
 
pose_image = mp_pose.Pose(static_image_mode=True, 
                          min_detection_confidence=0.5)
 
pose_video = mp_pose.Pose(static_image_mode=False, 
                          min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
 
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()

In [3]:
def detectPose(image_pose, pose, draw=False):
    original_image = image_pose.copy()
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    resultant = pose.process(image_in_RGB)
    if resultant.pose_landmarks and draw:    
        mp_drawing.draw_landmarks(image=original_image, 
                                  landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=2, circle_radius=2),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))
        #mp_drawing.plot_landmarks(resultant.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
            
    return original_image, resultant


In [4]:
import mathutils

def mediapipe_to_points(landmarks):
    # Define the conversion matrix to convert Mediapipe coordinates to Blender coordinates
    # The matrix flips the x and y axes and scales the coordinates to match Blender's scale
    conversion_matrix = mathutils.Matrix.Scale(-1, 4, (1, 0, 0)) @ mathutils.Matrix.Scale(-1, 4, (0, 1, 0)) @ mathutils.Matrix.Scale(1, 4, (0, 0, 1)) @ mathutils.Matrix.Scale(0.1, 4)

    # Create a list to store the 3D points
    points = []

    # Loop through the landmarks and transform each one to a 3D point
    for landmark in landmarks.pose_landmarks.landmark:
        # Transform the landmark to a Blender-compatible point using the conversion matrix
        point = mathutils.Vector((landmark.x, landmark.y, -landmark.z)) @ conversion_matrix

        # Add the point to the list of points
        points.append(point)

    # Return the list of points
    return points


In [5]:
# PREPARING IMAGE
image = cv2.imread("test_slika.jpg")
slika, res = detectPose(image, pose, True)
slika = cv2.cvtColor(slika, cv2.COLOR_BGR2RGB)


In [6]:
def convert_mediapipe_point(point, width, height):
    """
    Converts a Mediapipe point to world coordinates in Blender.
    
    Args:
        point: A tuple representing a single Mediapipe point, in the form (x, y).
        width: The width of the original video frame.
        height: The height of the original video frame.
        
    Returns:
        A Vector representing the world coordinates of the point in Blender.
    """
    # Convert the point from pixel coordinates to normalized coordinates.
    x = (point[0] / width) * 2 - 1
    y = (point[1] / height) * -2 + 1
    
    # Convert the normalized coordinates to world coordinates.
    camera = bpy.context.scene.camera
    matrix = camera.matrix_world.normalized().inverted()
    direction = Vector((x, y, -1.0))
    location = matrix @ direction
    return location

points = []
for point in res.pose_world_landmarks.landmark:
    points.append((point.x, point.y))

print(points)


[(0.012211071327328682, -0.6174448132514954), (0.026200205087661743, -0.6508697271347046), (0.026722656562924385, -0.6516284346580505), (0.026464838534593582, -0.6518865823745728), (-0.0054448554292321205, -0.6516978144645691), (-0.005085199140012264, -0.6524111032485962), (-0.005273385904729366, -0.6537255048751831), (0.08604583889245987, -0.6289809346199036), (-0.05969403684139252, -0.6366888284683228), (0.036169640719890594, -0.5900976061820984), (-0.006859739311039448, -0.5895802974700928), (0.16934728622436523, -0.46106940507888794), (-0.14584948122501373, -0.4793054163455963), (0.3485146462917328, -0.30545276403427124), (-0.321453332901001, -0.461068332195282), (0.3826168179512024, -0.14721012115478516), (-0.31484314799308777, -0.5897824764251709), (0.3864830732345581, -0.08107279986143112), (-0.2971324026584625, -0.6027514934539795), (0.34770095348358154, -0.08922240138053894), (-0.2810652256011963, -0.626030445098877), (0.3658641576766968, -0.13431906700134277), (-0.29862746596

In [ ]:
new_points = [points[9], points[2], points[4], points[6], points[3], points[5], points[7], points[8], points[0], points[11], points[13], points[15], points[10], points[12], points[14], points[16]]

In [7]:
import bpy
import mathutils

# Define the mediapipe landmarks and their corresponding bone names in Blender
landmarks = {
    0: 'Head',
    1: 'Neck',
    2: 'Right shoulder',
    3: 'Right elbow',
    4: 'Right wrist',
    5: 'Left shoulder',
    6: 'Left elbow',
    7: 'Left wrist',
    8: 'Hip',
    9: 'Right hip',
    10: 'Right knee',
    11: 'Right ankle',
    12: 'Left hip',
    13: 'Left knee',
    14: 'Left ankle',
    15: 'Right eye',
    16: 'Left eye',
    17: 'Right ear',
    18: 'Left ear'
}

# Define the Blender armature object and its bone names
armature_obj = bpy.data.objects['Armature']
bone_names = [bone.name for bone in armature_obj.data.bones]

# Define the 3D coordinates of the mediapipe landmarks
landmark_coords = [[0.0, 0.0, 0.0] for i in range(len(landmarks))]

# Loop through the mediapipe landmarks and get their 3D coordinates
for i in range(len(landmarks)):
    # Get the bone name for the current landmark
    bone_name = landmarks[i]
    # If the bone name exists in the Blender armature, get its 3D coordinates
    if bone_name in bone_names:
        # Get the Blender bone object and its 3D coordinates
        bone = armature_obj.pose.bones[bone_name]
        pos = bone.head
        # Convert the 3D coordinates from the bone's local space to the world space
        pos = bone.matrix @ mathutils.Vector(pos)
        landmark_coords[i] = [pos.x, pos.y, pos.z]

# Print the 3D coordinates of the mediapipe landmarks
print(landmark_coords)


AttributeError: 'NoneType' object has no attribute 'objects'

In [ ]:
import bpy
import mathutils

# Define the Mediapipe landmarks and their corresponding bone names in Blender
mp_landmarks = [
    ('nose', 'Nose'),
    ('left_eye_inner', 'LeftEyeInner'),
    ('left_eye', 'LeftEye'),
    ('left_eye_outer', 'LeftEyeOuter'),
    ('right_eye_inner', 'RightEyeInner'),
    ('right_eye', 'RightEye'),
    ('right_eye_outer', 'RightEyeOuter'),
    ('left_ear', 'LeftEar'),
    ('right_ear', 'RightEar'),
    ('mouth_left', 'MouthLeft'),
    ('mouth_right', 'MouthRight'),
    ('mouth_center', 'MouthCenter'),
    ('left_shoulder', 'LeftShoulder'),
    ('right_shoulder', 'RightShoulder'),
    ('left_elbow', 'LeftElbow'),
    ('right_elbow', 'RightElbow'),
    ('left_wrist', 'LeftWrist'),
    ('right_wrist', 'RightWrist'),
    ('left_pinky', 'LeftPinky'),
    ('right_pinky', 'RightPinky'),
    ('left_index', 'LeftIndex'),
    ('right_index', 'RightIndex'),
    ('left_thumb', 'LeftThumb'),
    ('right_thumb', 'RightThumb'),
    ('left_hip', 'LeftHip'),
    ('right_hip', 'RightHip'),
    ('left_knee', 'LeftKnee'),
    ('right_knee', 'RightKnee'),
    ('left_ankle', 'LeftAnkle'),
    ('right_ankle', 'RightAnkle'),
    ('left_heel', 'LeftHeel'),
    ('right_heel', 'RightHeel'),
    ('left_foot_index', 'LeftToeBase'),
    ('right_foot_index', 'RightToeBase')
]

# Define the 3D coordinates of the Mediapipe landmarks
landmark_coords = [[0.0, 0.0, 0.0] for i in range(len(mp_landmarks))]

# Loop through the Mediapipe landmarks and get their 3D coordinates
for i, landmark_name in enumerate(mp_landmarks):
    # Get the bone name for the current landmark
    bone_name = landmark_name[1]
    # Get the 3D coordinates of the current landmark
    pos = landmark_coords[i]
    # Create a new bone for the current landmark
    bone = bpy.data.armatures['Armature'].edit_bones.new(bone_name)
    # Set the head and tail positions of the bone
    bone.head = mathutils.Vector(pos)
    bone.tail = mathutils.Vector([pos[0], pos[1], pos[2]+0.1])
    # Connect the bone to its parent bone
    if i > 0:
        bone.parent = bpy.data.armatures['Armature'].edit_bones[mp_landmarks[i-1][1]]

# Create an armature object and link it to the scene
armature_obj = bpy.data.objects.new('Armature', bpy.data.armatures['Armature'])
bpy.context.collection.objects.link(armature_obj)

# Set the armature object as the active object
bpy.context.view_layer.objects.active = armature_obj

# Switch to pose mode and reset the armature to its rest position
bpy.ops.object.mode_set(mode='POSE')
bpy.ops.pose.select_all(action='SELECT')
bpy.ops.pose.transforms_clear()

# Switch back to object mode
bpy.ops.object.mode_set(mode='OBJECT')

# Print a message to indicate that


In [ ]:
import bpy

# Define the camera object
camera_obj = bpy.data.objects['Camera']

# Define the camera matrix and distortion coefficients
camera_matrix = camera_obj.data.camera_matrix
dist_coeffs = camera_obj.data.distortion_coefficients

# Define the Mediapipe landmarks and their corresponding 2D coordinates
mp_landmarks = [
    ('nose', [0.0, 0.0]),
    ('left_eye_inner', [0.0, 0.0]),
    ('left_eye', [0.0, 0.0]),
    ('left_eye_outer', [0.0, 0.0]),
    ('right_eye_inner', [0.0, 0.0]),
    ('right_eye', [0.0, 0.0]),
    ('right_eye_outer', [0.0, 0.0]),
    ('left_ear', [0.0, 0.0]),
    ('right_ear', [0.0, 0.0]),
    ('mouth_left', [0.0, 0.0]),
    ('mouth_right', [0.0, 0.0]),
    ('mouth_center', [0.0, 0.0]),
    ('left_shoulder', [0.0, 0.0]),
    ('right_shoulder', [0.0, 0.0]),
    ('left_elbow', [0.0, 0.0]),
    ('right_elbow', [0.0, 0.0]),
    ('left_wrist', [0.0, 0.0]),
    ('right_wrist', [0.0, 0.0]),
    ('left_pinky', [0.0, 0.0]),
    ('right_pinky', [0.0, 0.0]),
    ('left_index', [0.0, 0.0]),
    ('right_index', [0.0, 0.0]),
    ('left_thumb', [0.0, 0.0]),
    ('right_thumb', [0.0, 0.0]),
    ('left_hip', [0.0, 0.0]),
    ('right_hip', [0.0, 0.0]),
    ('left_knee', [0.0, 0.0]),
    ('right_knee', [0.0, 0.0]),
    ('left_ankle', [0.0, 0.0]),
    ('right_ankle', [0.0, 0.0]),
    ('left_heel', [0.0, 0.0]),
    ('right_heel', [0.0, 0.0]),
    ('left_foot_index', [0.0, 0.0]),
    ('right_foot_index', [0.0, 0.0])
]

# Define the 3D coordinates of the Mediapipe landmarks
landmark_coords = [[0.0, 0.0, 0.0] for i in range(len(mp_landmarks))]

# Loop through the Mediapipe landmarks and get their 3D coordinates
for i, landmark_name in enumerate(mp_landmarks):
    # Get the 2D coordinates of the current landmark
    x, y = landmark_name[1]
    # Get the 3D coordinates of the current landmark
    pos = camera_obj.matrix_world @ camera_matrix.inverted() @ [x, y, 1.0]
    # Add the 3D coordinates